# Buscando letras de músicas na Web
Nesse notebook, usei web scraping (com BeautifulSoup) e a API do Spotify para buscar as letras das músicas de determinado artista. A API do Spotify foi utilizada para pegar as músicas dos albuns dos artistas. A partir da lista de músicas, elas foram buscadas no site de letras [vagalume.com.br](https://www.vagalume.com.br). Como resultado, temos uma tabela com as letras das músicas a partir do nome do artista.

Para conseguir isso, foram realizados alguns passos, em formato de funções, resumidamente:
#### API do Spotify
 - Requisitar o token de acesso
 - Traduzir o nome do artista em um Id para localizá-lo na biblioteca Spotify
 - Requisitar os álbuns do artista
 - E as músicas de cada álbum
 
#### BeautifulSoup
 - Entrar na página do artista
 - Entrar na página de cada música
 - Raspar as letras

In [1]:
# Importar algumas bibliotecas

import pandas as pd
import requests                            # para realizar as requisições
import base64                              # para codificar em base64
import json                                # para ler os resultados
from config import clientID, clientSecret  # ID e Secret armazenados em outro arquivo
import re                                  # regex
import unidecode                           # para remover os acentos
from bs4 import BeautifulSoup              # para pegar as letras dos sites

In [2]:
# Função para pegar o token de acesso

def getToken():
    
    # Passar o ID e Secret para base64 para ir no header da requisição 
    message = f"{clientID}:{clientSecret}"
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii')
    
    authUrl = "https://accounts.spotify.com/api/token"        # endpoint da API
    authHeader = {'Authorization':'Basic '+ base64_message}   # header da requisição
    authData = {'grant_type':'client_credentials'}            # body
    
    res = requests.post(authUrl, headers=authHeader, data=authData)    # requisição POST, com header e body
    responseObject = res.json()                                        # resultado da requisição
    access_token = responseObject['access_token']                      # o token de acesso
    
    return access_token

In [3]:
# Função para pegar o id do artista a partir do nome

def getArtistId(artistName):
    
    name = artistName.replace(" ", "%20")   # substitui os espaços, para conseguir procurar
    
    access_token = getToken()               # token de acesso
    
    searchUrl = "https://api.spotify.com/v1/search?q=" + name + "&type=artist"  # endpoint da API
    header = {'Authorization': 'Bearer '+ access_token}                         # header da requisição
    
    res = requests.get(searchUrl, headers=header)   # requisição GET
    searchObject = res.json()                       # resultado da requisição
    
    artistId = searchObject['artists']['items'][0]['id']  # id do artista
    
    return artistId

In [4]:
# Função para pegar os albuns do artista a partir do nome

def getArtistAlbums(artistName):
    
    access_token = getToken()     # para todas as requisições da API do Spotify, precisa do token
    
    artistId = getArtistId(artistName)                                                       # ID do artista
    albunsUrl = f"https://api.spotify.com/v1/artists/{artistId}/albums?include_groups=album" # endpoint da API
    header = {'Authorization': 'Bearer '+ access_token}                                      # header da requisição
    
    res = requests.get(albunsUrl, headers=header)   # requisição GET
    albumsObject = res.json()                       # resultado da requisição
    
    # Salvar os albuns em um dataframe, com o nome e o Id
    albumNames = [album['name'] for album in albumsObject['items']]
    albumIds = [album['id'] for album in albumsObject['items']]
    
    albumsDf = pd.DataFrame({
        'albumNames' : albumNames,
        'albumIds' : albumIds
    })
    
    # Remove albuns ao vivo
    albumsDf = albumsDf[~albumsDf.albumNames.str.contains("Ao Vivo")]
    albumsDf = albumsDf[~albumsDf.albumNames.str.contains("Live")]
    
    # Remove duplicados
    albumsDf = albumsDf.drop_duplicates("albumNames")
    
    return albumsDf

In [5]:
# Função para pegar as músicas a partir do id do album

def getAlbumTracks(albumId):
    
    access_token = getToken()
    
    tracksUrl = f"https://api.spotify.com/v1/albums/{albumId}/tracks"   # endpoint da api
    header = {'Authorization': 'Bearer '+ access_token}                 # header da requisição

    res = requests.get(tracksUrl, headers=header)                       # requisição
    tracksObject = res.json()                                           # resultado da requisição
    
    trackNames = [track['name'] for track in tracksObject['items']]     # nomes das faixas armazenados nessa lista
    
    return trackNames

In [6]:
# Função para retornar as músicas do artista

def getArtistTracks(artistName):
    
    access_token = getToken()
    
    artistAlbums = getArtistAlbums(artistName)
    
    # Cria duas listas vazias, que vão ser populadas pelas músicas
    songsList = []
    albumsList = []

    # Itera os albuns, alimentando as listas acima
    for index, row in artistAlbums.iterrows():
        albumSongs = getAlbumTracks(artistAlbums['albumIds'][index])
        for i, r in enumerate(albumSongs):
            songsList.append(albumSongs[i])
            albumsList.append(row['albumNames'])

    # Cria um dataframe a partir das listas
    artistTracks = pd.DataFrame({
        'música': songsList,
        'album': albumsList
    })
    
    # Remove músicas ao vivo
    artistTracks = artistTracks[~artistTracks.música.str.contains("Ao Vivo")]
    artistTracks = artistTracks[~artistTracks.música.str.contains("Live")]
    
    # Remove músicas duplicadas
    artistTracks = artistTracks.drop_duplicates("música")
    
    return artistTracks

In [7]:
# Função para pegar o HTML da página do artista, que será lido para buscar as músicas

def getArtistHTML(artistName):
    
    baseUrl = "https://www.vagalume.com.br"
    
    name = artistName.replace(" ", "-").lower()    # troca espaço por - no nome, e deixa minúsculo
    
    artistUrl = baseUrl + "/" + name
    r = requests.get(artistUrl)                    # url para buscar o HTML 
    
    # Cria um objeto BeautifulSoup a partir do resultado do conteúdo da requisição
    artistHTML = BeautifulSoup(unidecode.unidecode(r.content.decode('utf-8', 'ignore')), 'html.parser')
        # Aqui tem que usar esse "unidecode" e "decode(utf-8)" para tirar os acentos 
    
    return artistHTML

In [8]:
# Função para pegar a letra dentro do HTML do artista, a partir do nome da música

def getSongLyric(songName, artistHTML):
    
    baseUrl = "https://www.vagalume.com.br"
    
    name = unidecode.unidecode(songName).lower()    # tira os acentos do nome, e deixa minúsculo
    
    # Procura pelo nome da música nas opções
    try:
        # Retorna o "href" do link da música
        relativePath = artistHTML.find("a", text=re.compile("(?i)" + name), attrs={"class": "nameMusic"})['href']

        songUrl = baseUrl + relativePath                                             # pegar a URL da música

        r = requests.get(songUrl)                                                    # acessar a URL da música
        songHTML = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser') # objeto BS a partir do conteúdo obtido

        # Retorna a letra da música (que está dentro de um div com id=lyrics)
        lyrics = songHTML.find("div", attrs={"id": "lyrics"} ).get_text(" ") 
   
    # Se não encontra, testa simplesmente com o nome da música na URL
    except TypeError:
        
        try:
            artist = artistName.replace(" ", "-").lower()

            relativePath = "/" + artist + "/" + name.replace(" ", "-")

            songUrl = baseUrl + relativePath                                            # possível URL da música

            r = requests.get(songUrl)                                                   # acessar a URL da música
            songHTML = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')

            lyrics = songHTML.find("div", attrs={"id": "lyrics"} ).get_text(" ")         # letra da música        
      
        # Se mesmo assim não encontra
        except AttributeError:
            lyrics = "Música não encontrada"
    
    return lyrics

In [9]:
# Função para pegar as letras do artista
def getLyrics(artistName):
    
    access_token = getToken()                   # token
    
    artistTracks = getArtistTracks(artistName)  # df com as músicas
    
    artistHTML = getArtistHTML(artistName)      # HTML do artista
    
    # Itera sobre as linhas do df, aplicando a função getSongLyric em cada uma delas, para retornas as letras
    artistTracks['letra'] = artistTracks.apply (lambda row: getSongLyric(row["música"], artistHTML), axis=1)
    
    artistLyrics = artistTracks.reset_index(drop=True)
    
    return artistLyrics

In [10]:
artistName = "armandinho"
result = getLyrics(artistName)

In [11]:
result.head()

,música,album,letra
0,Sol Loiro,Sol Loiro,"Te sinto tão longe, longe, longe, longe, mas t..."
1,A Ilha,Sol Loiro,"Eu sei, que existe uma ilha Onde o homem nunca..."
2,Trilha do Sol,Sol Loiro,Eu continuo com os pés na areia Continuo com m...
3,Acordar Cedo,Sol Loiro,Acordar cedo Sentir a energia salgada do vento...
4,Nada Mudou,Sol Loiro,Quando você estiver só E um raio de sol lembra...
